# Inference

In [ ]:
import torch
from IPython.display import display
from omegaconf import OmegaConf, open_dict
from ipywidgets import Select, HBox, Label
from nemo.collections.tts.models.base import SpectrogramGenerator, Vocoder

from nemo.collections.tts.models.fastpitch import FastPitchModel
from nemo.collections.tts.models import HifiGanModel

In [2]:
supported_spec_gen = ["tacotron2", "fastpitch", "mixertts", "mixerttsx", None]
supported_audio_gen = ["waveglow", "hifigan", "univnet", "griffin-lim", None]

print("Select the model(s) that you want to use. Please choose 1 spectrogram generator and 1 vocoder.")
spectrogram_generator_selector = Select(options=supported_spec_gen, value='fastpitch')
audio_generator_selector = Select(options=supported_audio_gen, value='hifigan')
display(HBox([spectrogram_generator_selector, Label("+"), audio_generator_selector]))

Select the model(s) that you want to use. Please choose 1 spectrogram generator and 1 vocoder.


In [3]:
spectrogram_generator = spectrogram_generator_selector.value
audio_generator = audio_generator_selector.value

if spectrogram_generator is None and audio_generator is None:
    raise ValueError("No models were chosen. Please return to the previous step and choose either 1 end-to-end model or 1 spectrogram generator and 1 vocoder.")

if (spectrogram_generator and audio_generator is None) or (audio_generator and spectrogram_generator is None):
    raise ValueError("In order to continue with the two step pipeline, both the spectrogram generator and the audio generator must be chosen, but one was `None`")

In [4]:
def load_spectrogram_model():
    override_conf = None
    
    from_pretrained_call = SpectrogramGenerator.from_pretrained
    
    if spectrogram_generator == "tacotron2":
        from nemo.collections.tts.models import Tacotron2Model
        pretrained_model = "tts_en_tacotron2"
    elif spectrogram_generator == "fastpitch":
        from nemo.collections.tts.models import FastPitchModel
        pretrained_model = "tts_en_fastpitch"
    elif spectrogram_generator == "mixertts":
        from nemo.collections.tts.models import MixerTTSModel
        pretrained_model = "tts_en_lj_mixertts"
    elif spectrogram_generator == "mixerttsx":
        from nemo.collections.tts.models import MixerTTSModel
        pretrained_model = "tts_en_lj_mixerttsx"
    else:
        raise NotImplementedError
        
    model = from_pretrained_call(pretrained_model, override_config_path=override_conf)
    
    return model

In [6]:
def load_vocoder_model():
    TwoStagesModel = False
    strict=True
    
    if audio_generator == "waveglow":
        from nemo.collections.tts.models import WaveGlowModel
        pretrained_model = "tts_waveglow"
        strict=False
    elif audio_generator == "hifigan":
        from nemo.collections.tts.models import HifiGanModel
        spectrogram_generator2ft_hifigan = {
            "mixertts": "tts_en_lj_hifigan_ft_mixertts",
            "mixerttsx": "tts_en_lj_hifigan_ft_mixerttsx"
        }
        pretrained_model = spectrogram_generator2ft_hifigan.get(spectrogram_generator, "tts_en_hifigan")
    elif audio_generator == "univnet":
        from nemo.collections.tts.models import UnivNetModel
        pretrained_model = "tts_en_lj_univnet"
    elif audio_generator == "griffin-lim":
        from nemo.collections.tts.models import TwoStagesModel
        cfg = {'linvocoder':  {'_target_': 'nemo.collections.tts.models.two_stages.GriffinLimModel',
                             'cfg': {'n_iters': 64, 'n_fft': 1024, 'l_hop': 256}},
               'mel2spec': {'_target_': 'nemo.collections.tts.models.two_stages.MelPsuedoInverseModel',
                           'cfg': {'sampling_rate': 22050, 'n_fft': 1024, 
                                   'mel_fmin': 0, 'mel_fmax': 8000, 'mel_freq': 80}}}
        model = TwoStagesModel(cfg)            
        TwoStagesModel = True
    else:
        raise NotImplementedError

    if not TwoStagesModel:
        model = Vocoder.from_pretrained(pretrained_model, strict=strict)
        
    return model

In [8]:
spec_gen = load_spectrogram_model().eval().cuda()
vocoder = load_vocoder_model().eval().cuda()

[NeMo I 2025-04-17 13:37:24 cloud:58] Found existing object /home/antonio/.cache/torch/NeMo/NeMo_2.1.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo.
[NeMo I 2025-04-17 13:37:24 cloud:64] Re-using file from: /home/antonio/.cache/torch/NeMo/NeMo_2.1.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo
[NeMo I 2025-04-17 13:37:24 common:826] Instantiating model from pre-trained checkpoint


[NeMo W 2025-04-17 13:37:24 experimental:26] `<class 'nemo.collections.tts.g2p.models.i18n_ipa.IpaG2p'>` is experimental and not ready for production yet. Use at your own risk.
[NeMo W 2025-04-17 13:37:26 i18n_ipa:124] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2025-04-17 13:37:26 experimental:26] `<class 'nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers.IPATokenizer'>` is experimental and not ready for production yet. Use at your own risk.
[NeMo W 2025-04-17 13:37:26 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.text_to_speech_dat

[NeMo I 2025-04-17 13:37:26 features:305] PADDING: 1
[NeMo I 2025-04-17 13:37:26 save_restore_connector:275] Model FastPitchModel was successfully restored from /home/antonio/.cache/torch/NeMo/NeMo_2.1.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo.
[NeMo I 2025-04-17 13:37:26 cloud:58] Found existing object /home/antonio/.cache/torch/NeMo/NeMo_2.1.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2025-04-17 13:37:26 cloud:64] Re-using file from: /home/antonio/.cache/torch/NeMo/NeMo_2.1.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2025-04-17 13:37:26 common:826] Instantiating model from pre-trained checkpoint


[NeMo W 2025-04-17 13:37:31 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2025-04-17 13:37:31 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2025-04-17 13:37:31 features:305] PADDING: 0


[NeMo W 2025-04-17 13:37:31 features:282] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2025-04-17 13:37:31 features:305] PADDING: 0
[NeMo I 2025-04-17 13:37:32 save_restore_connector:275] Model HifiGanModel was successfully restored from /home/antonio/.cache/torch/NeMo/NeMo_2.1.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [16]:
type(spec_gen), type(vocoder)

(nemo.collections.tts.models.fastpitch.FastPitchModel,
 nemo.collections.tts.models.hifigan.HifiGanModel)

In [26]:
def infer(spec_gen_model:FastPitchModel, vocoder_model:HifiGanModel, str_input:str):
    parser_model = spec_gen_model
    with torch.no_grad():
        parsed = parser_model.parse(str_input)
        gen_spec_kwargs = {'speaker': 3}
        
        if spectrogram_generator == "mixerttsx":
            gen_spec_kwargs["raw_texts"] = [str_input]
        
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, **gen_spec_kwargs)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
        if audio_generator == "hifigan":
            audio = vocoder_model._bias_denoise(audio, spectrogram).squeeze(1)
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

In [27]:
text_to_generate = input("Input what you want the model to say: ")
spec, audio = infer(spec_gen, vocoder, text_to_generate)

In [28]:
import IPython.display as ipd
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt

ipd.Audio(audio, rate=22050)